In [5]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

In [8]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi my name is Potato, I live in south Korea", "Wow that is so cool!")
add_message("South Korea is so pretty", "I wish I could go!!!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Korea from north Korea?", "I don't know! Super far!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Korea from north Korea?", "I don't know! Super far!")

get_history()


{'history': [SystemMessage(content='Potato from South Korea mentions the beauty of their country. The AI expresses a desire to visit and admits not knowing how far Korea is from Argentina. The human asks how far Korea is from North Korea. The AI responds that it is super far and the human then asks how far Korea is from Argentina.'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from north Korea?'),
  AIMessage(content="I don't know! Super far!")]}

In [9]:
add_message("Hi my name is Potato, I live in south Kores", "Wow that is so cool!")
add_message("South Kddorea is so pretty", "I wish I could go!!!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Korea from north Korea?", "I don't know! Super far!")
add_message("How far is Korea from Argentina?", "I don't know! Super far!")
add_message("How far is Korea from north Korea?", "I don't know! Super far!")
get_history()

{'history': [SystemMessage(content='Potato from South Korea mentions the beauty of their country. The AI expresses a desire to visit and admits not knowing how far Korea is from Argentina. The AI responds that it is super far. The human asks how far Korea is from North Korea, to which the AI replies that it is super far. The human introduces themselves as Potato from South Korea, and the AI responds with excitement at the coolness of the situation. The AI expresses a wish to go to Korea and the human asks how far Korea is from Argentina. The AI responds with uncertainty, saying it is super far. The human then asks how far Korea is from North Korea. The AI responds with "I don\'t know! Super far!" when asked about the distance between Korea and Argentina.'),
  AIMessage(content="I don't know! Super far!"),
  HumanMessage(content='How far is Korea from north Korea?'),
  AIMessage(content="I don't know! Super far!")]}